In [ ]:
import torch 
from torch.optim import lr_scheduler
from matplotlib import pyplot as plt
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l
import torchvision
from torchvision import models
import math
import numpy
d2l.use_svg_display()
device = torch.device('cuda')

In [ ]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)


In [ ]:
######定义VGG块
def vgg_block(num_convs, in_channels, out_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels,
                                kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2,stride=2))
    return nn.Sequential(*layers)
######定义NIN块
def nin_block(in_channels, out_channels, kernel_size, strides, padding):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, kernel_size, strides, padding),
        nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU(),
        nn.Conv2d(out_channels, out_channels, kernel_size=1), nn.ReLU())
######定义Inception块
class Inception(nn.Module):
    # c1--c4是每条路径的输出通道数
    def __init__(self, in_channels, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # 线路1，单1x1卷积层
        self.p1_1 = nn.Conv2d(in_channels, c1, kernel_size=1)
        # 线路2，1x1卷积层后接3x3卷积层
        self.p2_1 = nn.Conv2d(in_channels, c2[0], kernel_size=1)
        self.p2_2 = nn.Conv2d(c2[0], c2[1], kernel_size=3, padding=1)
        # 线路3，1x1卷积层后接5x5卷积层
        self.p3_1 = nn.Conv2d(in_channels, c3[0], kernel_size=1)
        self.p3_2 = nn.Conv2d(c3[0], c3[1], kernel_size=5, padding=2)
        # 线路4，3x3最大汇聚层后接1x1卷积层
        self.p4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.p4_2 = nn.Conv2d(in_channels, c4, kernel_size=1)

    def forward(self, x):
        p1 = F.relu(self.p1_1(x))
        p2 = F.relu(self.p2_2(F.relu(self.p2_1(x))))
        p3 = F.relu(self.p3_2(F.relu(self.p3_1(x))))
        p4 = F.relu(self.p4_2(self.p4_1(x)))
        # 在通道维度上连结输出
        return torch.cat((p1, p2, p3, p4), dim=1)


In [ ]:
class Residual(nn.Module):  #@save
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)
def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk
# b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
#                    nn.BatchNorm2d(64), nn.ReLU(),
#                    nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
# b2 = nn.Sequential(*resnet_block(64, 64, 2, first_block=True))
# b3 = nn.Sequential(*resnet_block(64, 128, 2))
# b5 = nn.Sequential(*resnet_block(128, 512, 2))
# little_resnet = nn.Sequential(b1, b2, b3, b5,
#                     nn.AdaptiveAvgPool2d((1,1)),
#                     nn.Flatten(), nn.Linear(512, 10))

In [ ]:
net = nn.Sequential(
    nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(), 
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
    nn.AvgPool2d(kernel_size=2, stride=2),
    nn.Flatten(),
    nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
    nn.Linear(120, 84), nn.Sigmoid(),
    nn.Linear(84, 10))




######使用ReLu
# net = nn.Sequential(
#     nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.ReLU(),
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Conv2d(6, 16, kernel_size=5), nn.ReLU(),
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Flatten(),
#     nn.Linear(16 * 5 * 5, 120), nn.ReLU(),
#     nn.Linear(120, 84), nn.ReLU(),
#     nn.Linear(84, 10))

#######添加批量标准化
# net = nn.Sequential(
#     nn.Conv2d(1, 6, kernel_size=5, padding=2), 
#     nn.BatchNorm2d(6),  
#     nn.Sigmoid(), 
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Conv2d(6, 16, kernel_size=5), 
#     nn.BatchNorm2d(16),  
#     nn.Sigmoid(),
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Flatten(),
#     nn.Linear(16 * 5 * 5, 120), 
#     nn.BatchNorm1d(120), 
#     nn.Sigmoid(),
#     nn.Linear(120, 84), 
#     nn.BatchNorm1d(84),  
#     nn.Sigmoid(),
#     nn.Linear(84, 10)
#     )

# ##########调整卷积大小
# net = nn.Sequential(
#     nn.Conv2d(1, 6, kernel_size=3, padding=1), nn.Sigmoid(), 
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Conv2d(6, 16, kernel_size=3,padding=2,stride=3), nn.Sigmoid(),
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Flatten(),
#     nn.Linear(16 * 3 * 3, 64), nn.Sigmoid(),
#     nn.Linear(64, 32), nn.Sigmoid(),
#     nn.Linear(32, 10))

###########改变输出通道数量
# net = nn.Sequential(
#     nn.Conv2d(1, 16, kernel_size=5, padding=2), nn.Sigmoid(), 
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Conv2d(16, 32, kernel_size=5), nn.Sigmoid(),
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Flatten(),
#     nn.Linear(32 * 5 * 5, 240), nn.Sigmoid(),
#     nn.Linear(240, 64), nn.Sigmoid(),
#     nn.Linear(64, 10))

########改变卷积层数——减少卷积层
# net = nn.Sequential(
#     nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(), 
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Flatten(),
#     nn.Linear(6 * 14 * 14, 120), nn.Sigmoid(),
#     nn.Linear(120, 84), nn.Sigmoid(),
#     nn.Linear(84, 10))
########改变卷积层数——增加卷积层
# net = nn.Sequential(
#     nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(), 
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Conv2d(16,16,kernel_size=3,padding=1),nn.Sigmoid(),
#     nn.AvgPool2d(kernel_size=2,stride=1),
#     nn.Flatten(),
#     nn.Linear(16 * 4 * 4, 120), nn.Sigmoid(),
#     nn.Linear(120, 84), nn.Sigmoid(),
#     nn.Linear(84, 10))

############使用MaxPooling
# net = nn.Sequential(
#     nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(), 
#     nn.MaxPool2d(kernel_size=2, stride=2),
#     nn.Conv2d(6, 16, kernel_size=5), nn.Sigmoid(),
#     nn.MaxPool2d(kernel_size=2, stride=2),
#     nn.Flatten(),
#     nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
#     nn.Linear(120, 84), nn.Sigmoid(),
#     nn.Linear(84, 10))

###########使用vgg块
# net = nn.Sequential(
#     vgg_block(num_convs=2,in_channels=1,out_channels=6), nn.Sigmoid(), 
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Conv2d(6, 16, kernel_size=5,padding=2,stride=2), nn.Sigmoid(),
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Flatten(),
#     nn.Linear(16 * 2 * 2, 32), nn.Sigmoid(),
#     nn.Linear(32, 16), nn.Sigmoid(),
#     nn.Linear(16, 10))

##########使用NIN块
# net = nn.Sequential(
#     nn.Conv2d(1, 6, kernel_size=5,stride=1, padding=2), nn.Sigmoid(), 
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nin_block(6, 16, kernel_size=5,strides=1,padding=0), nn.Sigmoid(),
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Flatten(),
#     nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
#     nn.Linear(120, 84), nn.Sigmoid(),
#     nn.Linear(84, 10))

##########使用Inception块
# net = nn.Sequential(
#     nn.Conv2d(1, 6, kernel_size=5, padding=2), nn.Sigmoid(), 
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     Inception(6,6,(12,6),(10,2),2), nn.Sigmoid(),
#     nn.AvgPool2d(kernel_size=2, stride=2),
#     nn.Flatten(),
#     nn.Linear(16 * 5 * 5, 120), nn.Sigmoid(),
#     nn.Linear(120, 84), nn.Sigmoid(),
#     nn.Linear(84, 10))



In [ ]:
def evaluate_accuracy_gpu(net, data_iter, device=None): #@save
    """使用GPU计算模型在数据集上的精度"""
    if isinstance(net, nn.Module):
        net.eval()  # 设置为评估模式
        if not device:
            device = next(iter(net.parameters())).device
    # 正确预测的数量，总预测的数量
    metric = d2l.Accumulator(2)
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(X, list):
                # BERT微调所需的（之后将介绍）
                X = [x.to(device) for x in X]
            else:
                X = X.to(device)
            y = y.to(device)
            metric.add(d2l.accuracy(net(X), y), y.numel())
    return metric[0] / metric[1]

In [ ]:
#@save
def train_ch6(net, train_iter, test_iter, num_epochs, lr, device):
    """用GPU训练模型(在第六章定义)"""
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    net.apply(init_weights)
    print('training on', device)
    net.to(device)
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    ####学习率指数衰减
    #scheduler = lr_scheduler.ExponentialLR(optimizer,0.8)
    ####学习率分段衰减
    #scheduler = lr_scheduler.StepLR(optimizer=optimizer,step_size=10,gamma=0.6,verbose=True)
    ####学习率余弦衰减
    #scheduler = lr_scheduler.CosineAnnealingLR(optimizer=optimizer,T_max=10,eta_min=0.01)

    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', xlim=[1, num_epochs],
                            legend=['train loss', 'train acc', 'test acc'])
    timer, num_batches = d2l.Timer(), len(train_iter)
    for epoch in range(num_epochs):
        # 训练损失之和，训练准确率之和，样本数
        metric = d2l.Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            timer.start()
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device)
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            with torch.no_grad():
                metric.add(l * X.shape[0], d2l.accuracy(y_hat, y), X.shape[0])
            timer.stop()
            train_l = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (train_l, train_acc, None))
        test_acc = evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
        
        ####使用学习率调度器
        #scheduler.step()
        
    print(f'loss {train_l:.3f}, train acc {train_acc:.3f}, '
          f'test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec '
          f'on {str(device)}')
        
        

In [ ]:
lr, num_epochs = 0.9, 50 
train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())
plt.show()

In [ ]:
##########查看网络中间层输出
import numpy as np
import matplotlib.pyplot as plt

def see_picture(x):
    if len(x.shape) == 4:
        # 多张图像，每张图像单独显示
        num_images = x.shape[1]
        for i in range(num_images):
            img = x[:, i].detach().squeeze().numpy()  # 去除多余的维度，并转换为NumPy数组
            plt.imshow(img)  # 使用灰度色彩映射
            plt.axis('off')  # 不显示坐标轴
            plt.show()
    elif len(x.shape) == 3:
        # 单张图像，直接显示
        img = x.detach().squeeze().numpy()  # 去除多余的维度，并转换为NumPy数组
        plt.imshow(img)  # 使用灰度色彩映射
        plt.axis('off')  # 不显示坐标轴
        plt.show()
    else:
        raise ValueError("Invalid input shape. Expected 3 or 4 dimensions.")
net.to('cpu')
net.eval()
x =train_iter.dataset[0][0]
x = x.unsqueeze(0)  # 添加一个维度作为批量大小
print(x.shape) 
see_picture(x)
conv_1 = net[0]
sigmoid_1 = net[1]
avgpool_1 = net[2]
conv_2 = net[3]
sigmoid_2 = net[4]
avgpool_2 = net[5]




In [ ]:
conv_1_out = conv_1(x)
print('经过第一个卷积层的输出为：')
see_picture(conv_1_out)

In [ ]:
sigmoid_1_out = sigmoid_1(conv_1_out)
print('经过第一个激活层的输出为：')
see_picture(sigmoid_1_out)

In [ ]:
avgpool_1_out = avgpool_1(sigmoid_1_out)
print('经过第一个池化层的输出为：')
see_picture(avgpool_1_out)

In [ ]:
conv_2_out = conv_2(avgpool_1_out)
print('经过第二个卷积层的输出为：')
see_picture(conv_2_out)

In [ ]:
sigmoid_2_out = sigmoid_2(conv_2_out)
print('经过第二个激活层的输出为：')
see_picture(sigmoid_2_out)

In [ ]:
avgpool_2_out = avgpool_2(sigmoid_2_out)
print('经过第二个池化层的输出为：')
see_picture(avgpool_2_out)